In this Notebook, we'll train a simple image classifier & deploy it as a web API.

We'll create an experiment.  We'll create a run within that experiment and train a model.

We'll then register the model.

We'll do that for two versions of the model.

Then we'll deploy the last version.

In [2]:
# import the Workspace class and check the azureml SDK version
import azureml.core
from azureml.core import Workspace

workspace = Workspace.from_config(path="/dbfs/aml_config/config.json")

print('Workspace name: ' + workspace.name, 
      'Azure region: ' + workspace.location, 
      'Subscription id: ' + workspace.subscription_id, 
      'Resource group: ' + workspace.resource_group, sep = '\n')

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

In [3]:
import os
import urllib.request

os.makedirs('./data', exist_ok = True)

urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', filename='./data/train-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', filename='./data/train-labels.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', filename='./data/test-images.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', filename='./data/test-labels.gz')

In [4]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import gzip
import numpy as np
import struct

# load compressed MNIST gz files and return numpy arrays
def load_data(filename, label=False):
    with gzip.open(filename) as gz:
        struct.unpack('I', gz.read(4))
        n_items = struct.unpack('>I', gz.read(4))
        if not label:
            n_rows = struct.unpack('>I', gz.read(4))[0]
            n_cols = struct.unpack('>I', gz.read(4))[0]
            res = np.frombuffer(gz.read(n_items[0] * n_rows * n_cols), dtype=np.uint8)
            res = res.reshape(n_items[0], n_rows * n_cols)
        else:
            res = np.frombuffer(gz.read(n_items[0]), dtype=np.uint8)
            res = res.reshape(n_items[0], 1)
    return res

# one-hot encode a 1-D array
def one_hot_encode(array, num_of_classes):
    return np.eye(num_of_classes)[array.reshape(-1)]

In [5]:
# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data('./data/train-images.gz', False) / 255.0
y_train = load_data('./data/train-labels.gz', True).reshape(-1)

X_test = load_data('./data/test-images.gz', False) / 255.0
y_test = load_data('./data/test-labels.gz', True).reshape(-1)

In [6]:
y_train

In [7]:
from azureml.core import Experiment

experiment_name = 'simple-classifier'
experiment = Experiment(workspace=workspace, name=experiment_name)
run = experiment.start_logging()

In [8]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

In [9]:
y_hat = clf.predict(X_test)
successRate = np.average(y_hat == y_test)

run.log("Success Rate", successRate)
print(successRate)

In [10]:
import pickle

# Write model to file
with open("LogisticRegression", "wb") as f:
  pickle.dump(clf,f)
run.upload_file(name = 'outputs/LogisticRegression', path_or_stream = 'LogisticRegression')

In [11]:
# Complete tracking and get link to details
run.complete()
print("Run completed")

In [12]:
model = run.register_model(model_name = "image-classifier", model_path = "outputs/LogisticRegression")

In [13]:
rootRun = experiment.start_logging()

In [14]:
# Regularization Rates
regs = [0.0001, 0.01, 0.1, 1, 2]

rootRun.log("Number of regularization rates", str(len(regs)))
bestSuccessRate=0
bestModel=None

for reg in regs:
  with rootRun.child_run("reg-" + str(reg)) as subRun:
    #  Train
    clf = LogisticRegression(C=reg)
    clf.fit(X_train, y_train)
    
    #  Test
    y_hat = clf.predict(X_test)
    successRate = np.average(y_hat == y_test)

    #  Log results
    subRun.log("Regularization rate", reg)
    subRun.log("Success Rate", successRate)
    rootRun.log("Success Rate", successRate)
    print('Regularization rate: ' + str(reg),
          'Success Rate: ' + str(successRate),
          sep='\n')
    
    # Write model to file
    with open("LogisticRegression", "wb") as f:
      pickle.dump(clf,f)
    subRun.upload_file(name = 'outputs/LogisticRegression', path_or_stream = 'LogisticRegression')
    
    # Keep best model
    if(successRate>bestSuccessRate):
      bestSuccessRate=successRate
      bestModel=clf

In [15]:
# Write model to file
with open("LogisticRegression", "wb") as f:
  pickle.dump(bestModel,f)
rootRun.upload_file(name = 'outputs/LogisticRegression', path_or_stream = 'LogisticRegression')

# Log metrics
rootRun.log("Best Success Rate", bestSuccessRate)
rootRun.log("Best Regularization Rate", bestModel.C)


In [16]:
# Complete tracking and get link to details
rootRun.complete()
print("Root Run completed")

In [17]:
model = rootRun.register_model(model_name = "image-classifier", model_path = "outputs/LogisticRegression")

Let's create a scoring script that computes an area of a circle, given the estimate within the pi_estimate model. The scoring script consists of two parts: 

 * The *init* method that loads the model. You can retrieve registered model using *Model.get_model_path* method. 
 * The *run* method that gets invoked when you call the web service. It computes the area of a circle using the well-known $area = \pi*radius^2$ formula. The inputs and outputs are passed as json-formatted strings.

In [19]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('image-classifier')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    print("Input:  " + str(data))
    # make prediction
    y_hat = model.predict(data)
    print ("Ouptut:  " + str(y_hat))
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

In [20]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())
with open("myenv.yml","r") as f:
    print(f.read())

In [21]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "MNIST",  "method" : "sklearn"}, 
                                               description='Predict MNIST with sklearn')

In [22]:
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

# Deploy
service = Webservice.deploy_from_model(workspace=workspace,
                                       name='image-classifier-svc',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

In [23]:
print(service.scoring_uri)

In [24]:
import json

# find 30 random samples from test set
n = 30
sample_indices = np.random.permutation(X_test.shape[0])[0:n]

test_samples = json.dumps({"data": X_test[sample_indices].tolist()})
test_samples = bytes(test_samples, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)

result